<a href="https://colab.research.google.com/github/tnc-br/ddf-insights-analytics/blob/main/origin_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Meta Data and Parameters**

In [ ]:
#@title Parameters
lat = -9
lon = -40
oxygen_measurements = [32,33,34]
nitrogen_measurements = [22,23,24]
carbon_measurements = [-20,-21,-23]

Requirements

In [ ]:
#Install requirements
# !pip install geemap


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 108.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 81.3 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11124 sha256=179254d0865da7eead1189ab89029c0e040e81fd9847cf41f916ecf456763195
  Store

Libraries

In [ ]:
#Import Libraries
import ee
import os
import numpy as np
import scipy


Authentication and Authorization

In [ ]:
#Authenticate
if os.path.isfile('key.json') :

    # connection to the service account
    service_account = 'fraud-detection-pipeline-sa@kazob-370920.iam.gserviceaccount.com'
    credentials = ee.ServiceAccountCredentials(service_account, 'key.json')
    ee.Initialize(credentials)
    print('initialized with key')

# if in local env use the local user credential
else:
    ee.Initialize()

initialized with key


Global Variable definition

In [ ]:
isoscapes_ee_path = 'users/kazob/isoscapes'

Datasets

In [ ]:
# import geemap
# Map = geemap.Map()
# #brazil_bounds = ee.FeatureCollection('FAO/GAUL/2015/level1').filter(ee.Filter.eq('ADM0_NAME', 'Brazil'))
# Map.addLayer(carbon_means_iso.select('b1'), {}, 'Brazil')
# #Map.centerObject(brazil_bounds,4)

Code

In [ ]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

fetching isoscapes

In [ ]:
parent_asset = ee.data.getAsset(isoscapes_ee_path)
parent_id = parent_asset['name']
parent_type = parent_asset['type']
asset_list = []
child_assets = ee.data.listAssets({'parent': parent_id})['assets']
for child_asset in child_assets:
    child_id = child_asset['name']
    child_type = child_asset['type']
    if child_type in ['FOLDER','IMAGE_COLLECTION']:
        # Recursively call the function to get child assets
        asset_list.extend(get_asset_list(child_id))
    else:
        asset_list.append(child_id)
asset_list

['projects/earthengine-legacy/assets/users/kazob/isoscapes/Brasil_Raster_Krig_SD_d15N',
 'projects/earthengine-legacy/assets/users/kazob/isoscapes/Raster_Brasil_krig_d15N',
 'projects/earthengine-legacy/assets/users/kazob/isoscapes/iso_d13C_map_wood_stack_crs',
 'projects/earthengine-legacy/assets/users/kazob/isoscapes/xgb_means_oxygen_isoscape',
 'projects/earthengine-legacy/assets/users/kazob/isoscapes/xgb_variances_oxygen_isoscape']

In [ ]:
for asset in asset_list:
  if ('oxygen' in asset) and ('mean' in asset):
    oxygen_means_iso = ee.Image(asset)
  elif 'oxygen' in asset and 'variance' in asset:
    oxygen_variances_iso = ee.Image(asset)
  elif 'd13C' in asset:
    carbon_means_iso = ee.Image(asset)
    carbon_variances_iso = ee.Image(asset).select('b1')
    print('moma')
  elif 'd15N' in asset and 'SD' in asset:
    nitrogen_variances_iso = ee.Image(asset)
  elif 'd15N' in asset and 'SD' not in asset:
    nitrogen_means_iso = ee.Image(asset)


moma


In [ ]:
#carbon_means_iso.getInfo()

POI definition

In [ ]:
poi = ee.Geometry.Point(lon,lat)

***Oxygen***

Isoscape

In [ ]:
isoscape_mean = oxygen_means_iso.reduceRegion(
  reducer = ee.Reducer.mean(),
  geometry = poi,
  scale = 30).getInfo().get('b1');
isoscape_var = oxygen_variances_iso.reduceRegion(
  reducer = ee.Reducer.mean(),
  geometry = poi,
  scale = 30).getInfo().get('b1');
if isoscape_var is not None:
  isoscape_std = np.sqrt(isoscape_var)
else:
  isoscape_std = 0

Isotope calculation

In [ ]:
isotope_mean = np.mean(oxygen_measurements)
isotope_std = np.std(oxygen_measurements)
print(isotope_mean,isotope_std, isoscape_mean, isoscape_std)



33.0 0.816496580927726 43.059565734863284 0.6551787351470115


ttest and p-value

In [ ]:
_, p_value_oxygen = scipy.stats.ttest_ind_from_stats(
            isotope_mean, isotope_std, len(oxygen_measurements), isoscape_mean, isoscape_std, 30, equal_var=False, alternative="two-sided")

**Carbon**

Isoscape

In [ ]:
###DELETE
carbon_means_iso = oxygen_means_iso
carbon_variances_iso = oxygen_variances_iso
#delete
isoscape_mean = carbon_means_iso.reduceRegion(
  reducer = ee.Reducer.mean(),
  geometry = poi,
  scale = 30).getInfo().get('b1');
isoscape_var = oxygen_variances_iso.reduceRegion(
  reducer = ee.Reducer.mean(),
  geometry = poi,
  scale = 30).getInfo().get('b1');
if isoscape_var is not None:
  isoscape_std = np.sqrt(isoscape_var)
else:
  isoscape_std = 0


Isotope calculation

In [ ]:
isotope_mean = np.mean(carbon_measurements)
isotope_std = np.std(carbon_measurements)
print(isotope_mean,isotope_std, isoscape_mean, isoscape_std)



-21.333333333333332 1.247219128924647 43.059565734863284 0.6551787351470115


ttest and p-value

In [ ]:
_, p_value_carbon = scipy.stats.ttest_ind_from_stats(
            isotope_mean, isotope_std, len(carbon_measurements), isoscape_mean, isoscape_std, 30, equal_var=False, alternative="two-sided")

**Nitrogen**

Isoscape

In [ ]:
isoscape_mean = nitrogen_means_iso.reduceRegion(
  reducer = ee.Reducer.mean(),
  geometry = poi,
  scale = 30).getInfo().get('b1');
isoscape_var = nitrogen_variances_iso.reduceRegion(
  reducer = ee.Reducer.mean(),
  geometry = poi,
  scale = 30).getInfo().get('b1');
if isoscape_var is not None:
  isoscape_std = np.sqrt(isoscape_var)
else:
  isoscape_std = 0

Isotope calculation

In [ ]:
isotope_mean = np.mean(nitrogen_measurements)
isotope_std = np.std(nitrogen_measurements)
print(isotope_mean,isotope_std, isoscape_mean, isoscape_std)



23.0 0.816496580927726 10.94708251953125 1.6114730492355358


ttest and p-value

In [ ]:
_, p_value_nitrogen = scipy.stats.ttest_ind_from_stats(
            isotope_mean, isotope_std, len(nitrogen_measurements), isoscape_mean, isoscape_std, 30, equal_var=False, alternative="two-sided")

Origin Verification

In [ ]:
origin_validity = p_value_oxygen * p_value_carbon * p_value_nitrogen